# Data Cleaning Notebook for Amazon E-Commerce Product Dataset

## Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Load the raw dataset
data_path = "../data/raw/Amazon-Products.csv"  # Adjust the file name/path if different
df = pd.read_csv(data_path)

In [3]:
# Display the first few rows and a summary of the data
print("Initial Dataset Preview:")
display(df.head())
print("\nDataset Info:")
df.info()
print("\nDataset Description:")
print(df.describe())
print("\nSummary of Missing Values:")
print(df.isnull().sum())
print("\nSummary of Duplicates:")
print(df.duplicated().sum())
print("\nSummary of Unique Values:")
print(df.nunique())
print("\nSummary of Data Types:")
print(df.dtypes)

Initial Dataset Preview:


,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"
3,3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990"
4,4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790"



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551585 entries, 0 to 551584
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      551585 non-null  int64 
 1   name            551585 non-null  object
 2   main_category   551585 non-null  object
 3   sub_category    551585 non-null  object
 4   image           551585 non-null  object
 5   link            551585 non-null  object
 6   ratings         375791 non-null  object
 7   no_of_ratings   375791 non-null  object
 8   discount_price  490422 non-null  object
 9   actual_price    533772 non-null  object
dtypes: int64(1), object(9)
memory usage: 42.1+ MB

Dataset Description:
          Unnamed: 0
count  551585.000000
mean     7006.200471
std      5740.835523
min         0.000000
25%      1550.000000
50%      5933.000000
75%     11482.000000
max     19199.000000

Summary of Missing Values:
Unnamed: 0             0
name               

## Handling Missing Values

### Ratings and No_of_Ratings Columns

In [4]:
# Fill missing and invalid values in 'ratings' and 'no_of_ratings' columns
def handle_ratings(row):
    try:
        # Attempt to convert ratings to float
        row['ratings'] = float(row['ratings']) if not pd.isnull(row['ratings']) else 0.0
    except (ValueError, TypeError):
        row['ratings'] = 0.0  # Set to 0 if conversion fails

    try:
        # Attempt to convert no_of_ratings to int
        row['no_of_ratings'] = int(row['no_of_ratings'].replace(',', '')) if not pd.isnull(row['no_of_ratings']) else 0
    except (ValueError, TypeError, AttributeError):
        row['no_of_ratings'] = 0  # Set to 0 if conversion fails

    # Handle cases where one column has value but the other is missing
    if pd.isnull(row['ratings']) or pd.isnull(row['no_of_ratings']):
        row['ratings'] = 0.0
        row['no_of_ratings'] = 0

    return row

# Apply the function to handle ratings and no_of_ratings
df = df.apply(handle_ratings, axis=1)

In [5]:
# Ensure data types are consistent
df['ratings'] = df['ratings'].astype(float)
df['no_of_ratings'] = df['no_of_ratings'].astype(int)

### Actual Price and Discounted Price Columns

In [6]:
# Ensure both columns contain only numerical data and convert to float
def clean_price_column(price):
    if isinstance(price, str):
        return float(price.replace('₹', '').replace(',', '').strip())
    elif pd.isnull(price):
        return np.nan
    else:
        return float(price)
    
df['discount_price'] = df['discount_price'].apply(clean_price_column)
df['actual_price'] = df['actual_price'].apply(clean_price_column)

In [7]:
# Handle missing values in 'discount_price' and 'actual_price' columns
def handle_prices(row):
    if pd.isnull(row['discount_price']) and pd.isnull(row['actual_price']):
        return None  # Mark for dropping
    elif pd.isnull(row['discount_price']):
        row['discount_price'] = row['actual_price']
    elif pd.isnull(row['actual_price']):
        row['actual_price'] = row['discount_price']

    if row['discount_price'] > row['actual_price']:
        # Swap values if discount price is greater than actual price
        row['discount_price'], row['actual_price'] = row['actual_price'], row['discount_price']

    if row['actual_price'] == 0:
        row['discount_price'], row['actual_price'] = 0.0001, 0.0001  # Set to a small value to avoid division by zero

    return row

df = df.apply(handle_prices, axis=1)

In [8]:
# Drop rows where both prices were missing
df.dropna(subset=['discount_price', 'actual_price'], inplace=True)

# Ensure data types are consistent
df['discount_price'] = df['discount_price'].astype(float)
df['actual_price'] = df['actual_price'].astype(float)

In [9]:
# Display updated information
print("\nUpdated Dataset Info:")
df.info()


Updated Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 533772 entries, 0 to 551584
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      533772 non-null  float64
 1   name            533772 non-null  object 
 2   main_category   533772 non-null  object 
 3   sub_category    533772 non-null  object 
 4   image           533772 non-null  object 
 5   link            533772 non-null  object 
 6   ratings         533772 non-null  float64
 7   no_of_ratings   533772 non-null  float64
 8   discount_price  533772 non-null  float64
 9   actual_price    533772 non-null  float64
dtypes: float64(5), object(5)
memory usage: 44.8+ MB


## Remove Duplicates

In [10]:
df = df.drop_duplicates()

## Create New Features

In [11]:
# Calculate discount_percentage
df['discount_percentage'] = ((df['actual_price'] - df['discount_price']) / df['actual_price']) * 100

# Create a feature for price difference
df['price_difference'] = df['actual_price'] - df['discount_price']

# Categorize products based on discount ranges
def categorize_discount(discount):
    if discount >= 50:
        return 'High Discount'
    elif discount >= 20:
        return 'Medium Discount'
    else:
        return 'Low Discount'

df['discount_category'] = df['discount_percentage'].apply(categorize_discount)

## Standardizing Text Columns

In [12]:
def clean_text(text):
    return text.lower().strip()

df['name'] = df['name'].apply(clean_text)
df['main_category'] = df['main_category'].apply(clean_text)
df['sub_category'] = df['sub_category'].apply(clean_text)

## Analyzing Ratings

In [13]:
# Create a feature to classify products as highly-rated or low-rated
def classify_rating(rating):
    if rating >= 4.0:
        return 'Highly Rated'
    elif rating >= 2.5:
        return 'Moderately Rated'
    elif rating < 2.5:
        return 'Low Rated'
    else: # Handle missing values
        return 'Unrated'

df['rating_category'] = df['ratings'].apply(classify_rating)

## Verifying and Saving the Cleaned Data

In [14]:
print("\nCleaned Dataset Info:")
df.info()


Cleaned Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 533772 entries, 0 to 551584
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           533772 non-null  float64
 1   name                 533772 non-null  object 
 2   main_category        533772 non-null  object 
 3   sub_category         533772 non-null  object 
 4   image                533772 non-null  object 
 5   link                 533772 non-null  object 
 6   ratings              533772 non-null  float64
 7   no_of_ratings        533772 non-null  float64
 8   discount_price       533772 non-null  float64
 9   actual_price         533772 non-null  float64
 10  discount_percentage  533772 non-null  float64
 11  price_difference     533772 non-null  float64
 12  discount_category    533772 non-null  object 
 13  rating_category      533772 non-null  object 
dtypes: float64(7), object(7)
memory usage: 61.1+ MB


In [15]:
print("\nCleaned Dataset Description:")
df.describe()


Cleaned Dataset Description:


,Unnamed: 0,ratings,no_of_ratings,discount_price,actual_price,discount_percentage,price_difference
count,533772.000000,533772.000000,533772.000000,5.337720e+05,5.337720e+05,533772.000000,5.337720e+05
mean,6986.472203,2.604223,567.528729,2.872724e+03,2.311128e+04,45.256477,2.023856e+04
std,5733.785479,1.892872,7108.018146,9.565776e+03,1.355082e+07,24.531764,1.355082e+07
min,0.000000,0.000000,0.000000,1.000000e-04,1.000000e-04,0.000000,0.000000e+00
25%,1544.000000,0.000000,0.000000,3.990000e+02,9.900000e+02,28.578233,3.000000e+02
50%,5913.000000,3.500000,5.000000,6.990000e+02,1.599000e+03,50.012503,7.000000e+02
75%,11452.000000,4.100000,51.000000,1.549000e+03,2.999000e+03,64.580726,1.548000e+03
max,19199.000000,5.000000,589547.000000,1.249990e+06,9.900000e+09,99.999994,9.899999e+09


In [16]:
print("\nSummary of Missing Values:")
df.isnull().sum()


Summary of Missing Values:


Unnamed: 0             0
name                   0
main_category          0
sub_category           0
image                  0
link                   0
ratings                0
no_of_ratings          0
discount_price         0
actual_price           0
discount_percentage    0
price_difference       0
discount_category      0
rating_category        0
dtype: int64

In [17]:
print("\nSummary of Duplicates:")
df.duplicated().sum()


Summary of Duplicates:


np.int64(0)

In [18]:
print("\nSummary of Unique Values:")
df.nunique()


Summary of Unique Values:


Unnamed: 0              19200
name                   383823
main_category              20
sub_category              112
image                  449207
link                   533772
ratings                    42
no_of_ratings            8191
discount_price          31045
actual_price            23170
discount_percentage    102256
price_difference        22836
discount_category           3
rating_category             3
dtype: int64

In [19]:
print("\nSummary of Data Types:")
df.dtypes


Summary of Data Types:


Unnamed: 0             float64
name                    object
main_category           object
sub_category            object
image                   object
link                    object
ratings                float64
no_of_ratings          float64
discount_price         float64
actual_price           float64
discount_percentage    float64
price_difference       float64
discount_category       object
rating_category         object
dtype: object

In [20]:
# Save the cleaned data to the processed folder
processed_data_path = "../data/processed/cleaned_amazon_products.csv"
df.to_csv(processed_data_path, index=False)
print(f"Cleaned data saved to {processed_data_path}")


Cleaned data saved to ../data/processed/cleaned_amazon_products.csv
